In [50]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%matplotlib notebook

In [18]:
main_url = 'https://www.snezak.si/plazovi'
avalanche_url = '/?date=season&startDate=&endDate='
c = requests.get(main_url+avalanche_url).content
soup = BeautifulSoup(c, 'lxml')

In [3]:
soup

<!DOCTYPE html>
<html class="no-js" lang="sl">
<head>
<title>Opaženi plazovi - Snežak</title>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="" name="description"/>
<meta content="" name="keywords"/>
<meta content="Opaženi plazovi" property="og:title"/>
<meta content="Snežak" property="og:site_name"/>
<meta content="https://www.snezak.si/plazovi/" property="og:url"/>
<meta content="" property="og:description"/>
<meta content="https://www.snezak.si/pic/snezak-fb.jpg" property="og:image"/>
<meta content="article" property="og:type"/>
<meta content="sl_SI" property="og:locale"/>
<link href="https://www.snezak.si/plazovi/" rel="canonical"/><link href="/common/css/style.3.css" rel="stylesheet" type="text/css"/>
<script>
window.q=[];window.$=function(f){ q.push(f) } // jquery
var x = document.querySelector.bind(document); // plain JS
var xx = document.querySelectorAll.bind(document); // plain JS
var language = 'sl';
</script>
<!-- Global site tag (gtag.js

In [57]:
locations = []
dates = []
participants = []
hurt_participants = []
dead_participants = []

for section in tqdm_notebook(soup.select('section.post-item')):
    specific_avalanche = section.select_one('a')['href']
    
    c = requests.get(main_url+specific_avalanche).content
    fresh_soup = BeautifulSoup(c, 'lxml')
    
    locations.append(fresh_soup.find('div', class_='route').text)
    dates.append(fresh_soup.find('time')['datetime'])
    
    for rate in fresh_soup.find_all('div', class_= 'rate'):
        n = rate.text.split()[-1]
        
        try:
            n = int(n)
        except ValueError:
            n = -1
            
        if rate.find(class_='title').text == 'Število udeleženih oseb':
            participants.append(n)
        if rate.find(class_='title').text == 'Število poškodovanih oseb':
            hurt_participants.append(n)
        if rate.find(class_='title').text == 'Število žrtev':
            dead_participants.append(n)

A Jupyter Widget

In [75]:
df = pd.DataFrame({'date': dates,
             'n': participants,
             'n_hurt': hurt_participants,
             'n_dead': dead_participants})

In [76]:
df[['region','location']]= pd.Series(locations).str.split(' - ', expand=True).loc[:,:1].apply(lambda x: x.str.strip())

In [77]:
df.head()

,date,n,n_dead,n_hurt,region,location
0,2018-04-21,0,0,0,Kamniško-Savinjske Alpe,"Planjava, Planjavske zelenice"
1,2018-04-21,-1,-1,-1,Julijske Alpe,Kopišča pod Dolkovo glavo
2,2018-04-21,-1,-1,-1,Julijske Alpe,s Križa v dolino pod Stenarskimi vratci
3,2018-04-20,-1,-1,-1,Julijske Alpe,Cmir JV pobočje
4,2018-04-17,-1,-1,-1,Julijske Alpe,"Bohinjsko-Tolminski greben, Vrh Planje, južno ..."
